# Quick Start

Get started with Lightning Rod SDK in just a few lines of code. This example demonstrates the simplest possible workflow: generating binary forecasting questions from news articles.

In [ ]:
%pip install lightningrod-ai

from IPython.display import clear_output
clear_output()

In [ ]:
import os
from datetime import datetime, timedelta
from lightningrod import (
    LightningRod,
    NewsSeedGenerator,
    QuestionGenerator,
    QuestionPipeline,
    WebSearchLabeler,
    AnswerType,
    AnswerTypeEnum,
)
from lightningrod._generated.models import QuestionRenderer

api_key = os.getenv("LIGHTNINGROD_API_KEY", "your-api-key-here")

client = LightningRod(api_key=api_key)

In [ ]:
answer_type = AnswerType(answer_type=AnswerTypeEnum.BINARY)

pipeline_config = QuestionPipeline(
    seed_generator=NewsSeedGenerator(
        start_date=datetime.now() - timedelta(days=30),
        end_date=datetime.now(),
        interval_duration_days=7,
        search_query="technology announcements",
    ),
    question_generator=QuestionGenerator(
        instructions="Generate forward-looking questions about technology announcements.",
        answer_type=answer_type,
    ),
    labeler=WebSearchLabeler(answer_type=answer_type),
    renderer=QuestionRenderer(answer_type=answer_type),
)

dataset = client.transforms.run(pipeline_config, max_questions=10)

In [ ]:
print(f"Generated dataset with {dataset.num_rows} samples")

samples = dataset.to_samples()
for i, sample in enumerate(samples[:3]):
    print(f"\nSample {i+1}:")
    if sample.question:
        print(f"  Question: {sample.question.question_text}")
    if sample.label:
        print(f"  Answer: {sample.label.label}")
        print(f"  Confidence: {sample.label.label_confidence}")